# FIT5148 Assignment 2
#### Task C1, Part a

* Due: 24/05/2019
* Tutor: Paras Sitoula, Wednesday 12-2pm
<br>

| Student 1 | Student 2|
|-----------|----------|
| Hitesh Get | Samuel Campbell |

## 1. Import Necessary Modules

In [7]:
import os
import csv
import random
from time import sleep
import json
import geohash as gh
from kafka import KafkaProducer
import random
from datetime import datetime
from pprint import pprint

## 2. Set Working Directory

In [8]:
print("Please enter a valid path containing the climate and hotspot data")
os.chdir(input())

Please enter a valid path containing the climate and hotspot data


## 3. Read Climate Data

In [9]:
with open('./climate_streaming.csv') as file:
    climate_stream = list(csv.reader(file))

In [10]:
print("For your convenience, please find an example JSON-formatted climate record below:")
pprint({
    climate_stream[0][0]: climate_stream[1][0],
    climate_stream[0][1]: climate_stream[1][1],
    climate_stream[0][2]: climate_stream[1][2],
    climate_stream[0][3]: climate_stream[1][3],
    climate_stream[0][4]: climate_stream[1][4],
    climate_stream[0][5]: climate_stream[1][5],
    climate_stream[0][6]: climate_stream[1][6]
})

For your convenience, please find an example JSON-formatted climate record below:
{'air_temperature_celcius': '19',
 'latitude': '-37.623',
 'longitude': '149.323',
 'max_wind_speed': '11.1',
 'precipitation ': ' 0.00I',
 'relative_humidity': '56.8',
 'windspeed_knots': '7.9'}


## 4. Instantiate Producer

In [ ]:
def publish_message(producer_instance, topic_name, key, data):
    """
    This publishing function encodes the data according to the provided `key` (here json-formatted) 
    and character encoding format (here utf-8).
    
    Then, the data are broadcast to the server with the provided `topic_name`.
    """
    try:
        key_bytes = bytes(key, encoding='utf-8')
        producer_instance.send(topic_name, key = key_bytes, value=data)
        producer_instance.flush()
        print('Message published successfully.  ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print("Publish MEssage",str(ex))        

def connect_kafka_producer():
    """
    Instantiates a producer on localhost.
    """
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  value_serializer = lambda x: json.dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    
if __name__ == '__main__':
    # Set topic
    topic = 'climate'
    
    print('Publishing records..')
    # Connect the producer
    producer = connect_kafka_producer()
      
    # JSON-format each randomly-selected record and push it to the stream
    for i in random.sample(range(1, len(climate_stream)), len(climate_stream) - 1):
        climate_stream_1 = dict()
        climate_stream_1["Climate"] = {}
        climate_stream_1["Climate"]["latitude"] = float(climate_stream[i][0])
        climate_stream_1["Climate"]["longitude"] = float(climate_stream[i][1])
        climate_stream_1["Climate"]["air_temperature_celcius"] = climate_stream[i][2]
        climate_stream_1["Climate"]["relative_humidity"] = climate_stream[i][3]
        climate_stream_1["Climate"]["windspeed_knots"] = climate_stream[i][4]
        climate_stream_1["Climate"]["max_wind_speed"] = climate_stream[i][5]
        climate_stream_1["Climate"]["precipitation"] = climate_stream[i][6]
        climate_stream_1["Time_Stamp"] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        climate_stream_1["sender_id"] = "climate_streamer"
        climate_stream_1["station"] = "climate_station"
        climate_stream_1['hotspot'] = []
        climate_stream_1['geohash'] = gh.encode(climate_stream_1['Climate']["latitude"], climate_stream_1['Climate']["longitude"], precision = 5)
        data = climate_stream_1
        publish_message(producer, topic, 'jsondata', data)
        
        sleep(5)